In [3]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [4]:
from companies.models import Company

ImproperlyConfigured: Requested setting INSTALLED_APPS, but settings are not configured. You must either define the environment variable DJANGO_SETTINGS_MODULE or call settings.configure() before accessing settings.

In [5]:
Company.objects.create(name="Company AAA", num_employees=120, num_chairs=150, num_tables=60)
Company.objects.create(name="Company BBB", num_employees=50, num_chairs=30, num_tables=20)
Company.objects.create(name="Company CCC", num_employees=100, num_chairs=40, num_tables=40)

<Company: Company CCC>

In [7]:
from django.db.models import Count, F, Value
from django.db.models.functions import Length, Upper
from django.db.models.lookups import GreaterThan

In [10]:
# Find companies that have more employees than chairs.
qs = Company.objects.filter(num_employees__gt=F("num_chairs"))

In [11]:
print(qs.query)

SELECT "companies_company"."id", "companies_company"."name", "companies_company"."ticker", "companies_company"."num_employees", "companies_company"."num_tables", "companies_company"."num_chairs" FROM "companies_company" WHERE "companies_company"."num_employees" > ("companies_company"."num_chairs")


In [12]:
# Find companies that have at least twice as many employees as chairs.
qs = Company.objects.filter(num_employees__gt=F("num_chairs") * 2)

In [13]:
print(qs.query)

SELECT "companies_company"."id", "companies_company"."name", "companies_company"."ticker", "companies_company"."num_employees", "companies_company"."num_tables", "companies_company"."num_chairs" FROM "companies_company" WHERE "companies_company"."num_employees" > ("companies_company"."num_chairs" * 2)


## Annotation

In [19]:
# How many chairs are needed for each company to seat all employees?
company = (
    Company.objects.filter(num_employees__gt=F("num_chairs"))
    .annotate(chairs_needed=F("num_employees") - F("num_chairs"))
)

In [20]:
for c in company:
    print(f"{c.name}: Chair Needed = {c.chairs_needed}")

Company BBB: Chair Needed = 20
Company CCC: Chair Needed = 60


In [26]:
company = Company.objects.annotate(chairs_needed=F("num_employees") - F("num_chairs")).filter(chairs_needed__gt=50)


In [27]:
for c in company:
    print(f"{c.name}: Chair Needed = {c.chairs_needed}")

Company CCC: Chair Needed = 60


In [28]:
print(company.query)

SELECT "companies_company"."id", "companies_company"."name", "companies_company"."ticker", "companies_company"."num_employees", "companies_company"."num_tables", "companies_company"."num_chairs", ("companies_company"."num_employees" - "companies_company"."num_chairs") AS "chairs_needed" FROM "companies_company" WHERE ("companies_company"."num_employees" - "companies_company"."num_chairs") > 50


In [35]:
company = Company.objects.create(name="Google", ticker=Upper(Value("goog")), num_employees=10, num_tables=20, num_chairs=30)

In [38]:
# Order By
Company.objects.all().order_by("-name", "num_employees")

<QuerySet [<Company: Google>, <Company: Company CCC>, <Company: Company BBB>, <Company: Company AAA>]>

In [39]:
Company.objects.all().order_by(Length("name").asc())

<QuerySet [<Company: Google>, <Company: Company AAA>, <Company: Company BBB>, <Company: Company CCC>]>

In [40]:
Company.objects.all().order_by(Length("name").desc())

<QuerySet [<Company: Company AAA>, <Company: Company BBB>, <Company: Company CCC>, <Company: Google>]>

In [41]:
# Lookup expressions can also be used directly in filters
Company.objects.filter(GreaterThan(F("num_employees"), F("num_chairs")))
# or annotations.
Company.objects.annotate(
    need_chairs=GreaterThan(F("num_employees"), F("num_chairs")),
)

<QuerySet [<Company: Company AAA>, <Company: Company BBB>, <Company: Company CCC>, <Company: Google>]>

# Books

In [1]:
from books.models import Book

ImproperlyConfigured: Requested setting INSTALLED_APPS, but settings are not configured. You must either define the environment variable DJANGO_SETTINGS_MODULE or call settings.configure() before accessing settings.

In [2]:
# Total number of books.
Book.objects.count()

NameError: name 'Book' is not defined

In [6]:
# Total number of books with publisher=Penguin Books
Book.objects.filter(publisher__name="Penguin Books").count()

20

In [7]:
Book.objects.aggregate(Avg("price", default=0))

{'price__avg': Decimal('9.7018644067796610')}

In [8]:
Book.objects.aggregate(Max("price", default=0))

{'price__max': Decimal('14.99')}

In [ ]:
Book.objects.annotate(avg_price=)